## Pattern matching with regular expressions

In this chapter you'll learn about regular expressions, a language for describing patterns in strings. By combining regular expressions with the stringr functions you'll greatly increase your power to manipulate strings.

### Matching the start or end of the string
rebus provides START and END shortcuts to specify regular expressions that match the start and end of the string. These are also known as anchors. You can try it out just by typing

START

You'll see the output <regex> ^. The <regex> denotes this is a special regex object and it has the value ^. ^ is the character used in the regular expression language to denote the start of a string.

The special operator provided by rebus, %R% allows you to compose complicated regular expressions from simple pieces. When you are reading rebus code, think of %R% as "then". For example, you could combine START with c,

START %R% "c"

to match the pattern "the start of string then a c", or in other words: strings that start with c. In rebus, if you want to match a specific character, or a specific sequence of characters, you simply specify them as a string, e.g. surround them with ".

In [1]:
library(stringr)
# install.packages("rebus")
library(rebus)
# Some strings to practice with
x <- c("cat", "coat", "scotland", "tic toc")

# Which character is used to match the end of a string? Print END to find out.
print (END)

# Run me
str_view(x, pattern = START %R% "c")

Warning message:
"package 'rebus' was built under R version 3.6.3"
Attaching package: 'rebus'

The following object is masked from 'package:stringr':

    regex



<regex> $


HTML widgets cannot be represented in plain text (need html)

In [2]:
# Match the strings that start with "co" 
str_view(x, pattern = START %R% "co")

HTML widgets cannot be represented in plain text (need html)

In [3]:
# Provide the pattern argument to match strings that end in "at". 
# Think about this as matching an "at" followed by the end of the string.

# Match the strings that end with "at"
str_view(x, pattern = "at" %R% END)

HTML widgets cannot be represented in plain text (need html)

In [4]:
# Provide the pattern argument to match strings that are "cat" exactly. 
# Think about this as matching the start of the string, followed by "cat" and then the end of the string.

# Match the string that is exactly "cat"
str_view(x, pattern = START %R% "cat" %R% END)

HTML widgets cannot be represented in plain text (need html)

### Matching any character
In a regular expression you can use a wildcard to match a single character, no matter what the character is. In rebus it is specified with ANY_CHAR. Try typing ANY_CHAR in the console. You should see that in the regular expression language this is specified by a dot, ..

For example, "c" %R% ANY_CHAR %R% "t" will look for patterns like "c_t" where the blank can be any character. Consider the strings: "cat", "coat", "scotland" and "tic toc". Where would the matches to "c" %R% ANY_CHAR %R% "t" be?

Test your intuition by running:

str_view(c("cat", "coat", "scotland", "tic toc"), 
  pattern = "c" %R% ANY_CHAR %R% "t")

Notice that ANY_CHAR will match a space character (c t in tic toc). It will also match numbers or punctuation symbols, but ANY_CHAR will only ever match one character, which is why we get no match in coat.

In [5]:
# Match two characters, where the second is a "t"
str_view(x, pattern = ANY_CHAR %R% "t")

HTML widgets cannot be represented in plain text (need html)

In [6]:
# Match a "t" followed by any character
str_view(x, pattern = "t" %R% ANY_CHAR)

HTML widgets cannot be represented in plain text (need html)

In [7]:
# Match any two characters. Notice the first two characters are matched. 
# Regular expressions are lazy and will take the first match they find.

# Match two characters
str_view(x, pattern = ANY_CHAR %R% ANY_CHAR)


HTML widgets cannot be represented in plain text (need html)

In [8]:
# Match a string with exactly three characters
str_view(x, pattern = START %R% ANY_CHAR %R% ANY_CHAR %R% ANY_CHAR %R% END)

## Note the difference: EXACTLY with start and end, and ANY without start and end

HTML widgets cannot be represented in plain text (need html)

### Combining with stringr functions
You can pass a regular expression as the pattern argument to any stringr function that has the pattern argument. You can use str_detect() to get a logical vector for whether there was a match, str_subset() to return just the strings with matches, and str_count() to count the number of matches in each string.

As a reminder, compare the output of those three functions with our "c_t" pattern from the previous exercise:

x <- c("cat", "coat", "scotland", "tic toc")
pattern <- "c" %R% ANY_CHAR %R% "t"
str_detect(x, pattern)
str_subset(x, pattern)
str_count(x, pattern)
It now also makes sense to add str_extract() to your repertoire. It returns just the part of the string that matched the pattern:

str_extract(x, pattern)
You'll combine your regular expression skills with stringr to ask how often a q is followed by any character in boy names.

It's always a good idea to test your pattern, so this pattern is shown matched with four names. The first two shouldn't have matches (can you explain why?) but the last two should.

In [9]:
library(babynames)
library(dplyr)

# Extracting vectors for boys' and girls' names
babynames_2014 <- filter(babynames, year == 2014)
boy_names <- filter(babynames_2014, sex == "M")$name
girl_names <- filter(babynames_2014, sex == "F")$name

# Find the boy_names with the pattern by using str_subset(). Assign the result to names_with_q.

pattern <- "q" %R% ANY_CHAR

# Find names that have the pattern
names_with_q <- str_subset(boy_names, pattern = pattern)

# How many names were there?
length(names_with_q)

Warning message:
"package 'babynames' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



[1] 96

In [19]:
# Find just the part of boy_names that matched with str_extract(). Assign the result to part_with_q.
# Find part of name that matches pattern
part_with_q <- str_extract(boy_names, pattern = pattern)

# Get a table of counts
table(part_with_q)

part_with_q
qa qe qi qm qo qu 
 1  1  2  2  1 89 

In [20]:
# Check that there weren't any boy_names that might have had the pattern twice (you would have only found the first match)

# Did any names have the pattern more than once?
count_of_q <- str_count(boy_names, pattern)

# Get a table of counts
table(count_of_q)

count_of_q
    0     1 
13951    96 

In [21]:
# Get a logical vector of whether or not each boy's name contains q* by calling str_detect(). Assign the result to with_q.

# Which babies got these names?
with_q <- str_detect(boy_names, pattern = pattern)

# What fraction of babies got these names?
mean(with_q)

[1] 0.006834199

### Alternation
The rebus function or() allows us to specify a set of alternatives, which may be single characters or character strings, to be matched. Each alternative is passed as a separate argument.

For example, or("grey", "gray") allows us to detect either the American or British spelling:

x <- c("grey sky", "gray elephant")
str_view(x, pattern = or("grey", "gray"))
Since these two words only differ by one character you could equivalently specify this match with "gr" %R% or("e", "a") %R% "y", that is "a gr followed by, an e or an a, then a y".

Notice we've added the argument match = TRUE to str_view(), this will only display elements that had a match, which is useful when you are searching over many strings.

In [22]:
#Create a pattern that matches Jeffrey or Geoffrey in boy_names. Assign this to whole_names.
whole_names <- or("Jeffrey", "Geoffrey")
str_view(boy_names, pattern = whole_names, match = TRUE)

HTML widgets cannot be represented in plain text (need html)

In [23]:
# Create a pattern that matches Je or Geo followed by ffrey. Assign this to common_ending. 
# You should get the same matches as the previous pattern.

# Match Jeffrey or Geoffrey, another way
common_ending <- or("Je", "Geo") %R% "ffrey"
str_view(boy_names, pattern = common_ending, match = TRUE)

HTML widgets cannot be represented in plain text (need html)

In [24]:
# Create a pattern that matches Je or Geo, followed by ff, followed by one of ry, ery rey, or erey. Assign this to by_parts.

by_parts <- or("Je", "Geo") %R% "ff" %R% or("ry", "ery", "rey", "erey") 
str_view(boy_names, pattern = by_parts, match = TRUE)

HTML widgets cannot be represented in plain text (need html)

In [27]:
# Create a pattern that matches all girl_names that start with C or K followed by ath. Assign this to ckath.

ckath <- START %R% or("C", "K") %R% "ath"
str_view(girl_names, pattern = ckath, match = TRUE)

HTML widgets cannot be represented in plain text (need html)

### Character classes
In regular expressions a character class is a way of specifying "match one (and only one) of the following characters". In rebus you can specify the set of allowable characters using the function char_class().

This is another way you could specify an alternate spelling, for example, specifying "a gr followed by, either an a or e, followed by a y":

x <- c("grey sky", "gray elephant")
str_view(x, pattern = "gr" %R% char_class("ae") %R% "y")
A negated character class matches "any single character that isn't one of the following", and in rebus is specified with negated_char_class().

Unlike in other places in a regular expression you don't need to escape characters that might otherwise have a special meaning inside character classes. If you want to match . you can include . directly, e.g. char_class("."). Matching a - is a bit trickier. If you need to do it, just make sure it comes first in the character class.

In [29]:
# Create a character class that contains vowels, a, e, i, o, u and their upper case versions.

vowels <- char_class("aeiou","AEIOU")

# Print vowels. In the regular expression language, a character class is put inside [.

print(vowels)

# View the matches to the pattern vowels in x with str_view(). Notice how only the first vowel is matched.

str_view(x, pattern = vowels)

<regex> [aeiouAEIOU]


HTML widgets cannot be represented in plain text (need html)

In [30]:
# View the matches to the pattern vowels in x with str_view_all(). Now all matches are highlighted.

str_view_all(x, pattern = vowels)

HTML widgets cannot be represented in plain text (need html)

In [33]:
# Find the number of vowels in each boy_names by combining str_count() with the vowels pattern.

num_vowels <- str_count(boy_names, pattern = vowels)

# Find the number of characters in each boy_names with str_length().

name_length <- str_length(boy_names)

# Find the average number of vowels in boy_names using the mean of num_vowels.
mean(num_vowels)

# Calc mean fraction of vowels per name
mean(num_vowels/ name_length)

[1] 2.385563

[1] 0.4000596

### Repetition
The rebus functions one_or_more(), zero_or_more() and optional() can be used to wrap parts of a regular expression to allow a pattern to match a variable number of times.

Take our vowels pattern from the last exercise. You can pass it to one_or_more() to create the pattern that matches "one or more vowels". Take a look with these interjections:

x <- c("ow", "ooh", "yeeeah!", "shh")
str_view(x, pattern = one_or_more(vowels))
You'll see we can match the single o in ow, the double o in ooh and the string of es followed by the a in yeeeah, but nothing in shh because there isn't a single vowel.

In contrast zero_or_more() will match even if there isn't an occurrence, try

str_view(x, pattern = zero_or_more(vowels))
Since both yeeeah and shh start without a vowel, they match "zero vowels", and since regular expressions are lazy, they look no further and return the start of the string as a match.

In [35]:
# Find boy_names that are only vowels, by combining one_or_more() with vowels. 
# You will need to either specify START and END or use exactly().

# Vowels from last exercise
vowels <- char_class("aeiouAEIOU")

# See names with only vowels
str_view(boy_names, 
  pattern = exactly(one_or_more(vowels)), 
  match = TRUE)

HTML widgets cannot be represented in plain text (need html)

In [36]:
# Use negated_char_class() for everything but vowels
not_vowels <- negated_char_class("aeiouAEIOU")

# See names with no vowels
str_view(boy_names, 
  pattern = exactly(one_or_more(not_vowels)), 
  match = TRUE)

HTML widgets cannot be represented in plain text (need html)

### Hunting for phone numbers
For your first task you are going to pull out the phone numbers from this vector of contact information:

contact
You'll assume the phone numbers you are looking for follow the American standard of a three digit area code, a three digit exchange and then a four digit number, but each part could be separated by spaces or various punctuation symbols.

In [42]:
contact = c("Call me at 555-555-0191", "123 Main St", "(555) 555 0191", "Phone: 555.555.0191 Mobile: 555.555.0192")

# Create a three digit pattern
three_digits <- DGT %R% DGT %R% DGT

# Test it
str_view_all(contact, pattern = three_digits)

HTML widgets cannot be represented in plain text (need html)

In [43]:
# There might be a range of separators, so make a separator pattern, that uses char_class() 
# to combine a character class containing -, ., (, ), and .

# Create a separator pattern
separator <- char_class("0-9")

# Test it
str_view_all(contact, pattern = separator)

HTML widgets cannot be represented in plain text (need html)

In [44]:
# Put together a pattern to match a whole phone number: an optional open parenthesis (OPEN_PAREN), 
# followed by three digits, followed by zero or more separators, followed by three digits followed 
# by zero or more separators, followed by four digits.

# Use these components
three_digits <- DGT %R% DGT %R% DGT
four_digits <- three_digits %R% DGT
separator <- char_class("-.() ")

# Create phone pattern
phone_pattern <- optional(OPEN_PAREN) %R%
  three_digits %R%
  zero_or_more(separator) %R%
  three_digits %R% 
  zero_or_more(separator) %R%
  four_digits
        
# Test it           
str_view_all(contact, pattern = phone_pattern)

HTML widgets cannot be represented in plain text (need html)

In [45]:
# Use this pattern
three_digits <- DGT %R% DGT %R% DGT
four_digits <- three_digits %R% DGT
separator <- char_class("-.() ")
phone_pattern <- optional(OPEN_PAREN) %R% 
  three_digits %R% 
  zero_or_more(separator) %R% 
  three_digits %R% 
  zero_or_more(separator) %R%
  four_digits
  
# Extract phone numbers
str_extract(contact, pattern = phone_pattern)

# Extract ALL phone numbers
str_extract_all(contact, pattern = phone_pattern)

[1] "555-555-0191"   NA               "(555) 555 0191" "555.555.0191"

[[1]]
[1] "555-555-0191"

[[2]]
character(0)

[[3]]
[1] "(555) 555 0191"

[[4]]
[1] "555.555.0191" "555.555.0192"

### Extracting age and gender from accident narratives
Recall from the video, you want to parse out age and gender from accident narratives. For example, this narrative

19YOM-SHOULDER STRAIN-WAS TACKLED WHILE PLAYING FOOTBALL W/ FRIENDS 
describes male of age 19, and this one

TRIPPED OVER CAT AND LANDED ON HARDWOOD FLOOR. LACERATION ELBOW, LEFT. 33 YOF*
a female of age 33.

You are generally looking for a pattern with a number, something to indicate the units, e.g. YO or YR for years old, or MO for months old, and a character that identifies the gender.

In this exercise you'll build up a pattern to pull out the part of the narrative that has the age and gender information. Then, in the next exercise you'll parse out the age and gender into separate variables.

In [48]:
# load narrative rds file
narr <- readRDS(file = "narratives.rds")

# Create an age pattern that matches one or two digits. Test your pattern using str_view().

# Pattern to match one or two digits
age <- one_or_more(DGT %R% DGT)

# Test it
str_view(narr, pattern = age)

HTML widgets cannot be represented in plain text (need html)

In [51]:
# Use this pattern
age <- DGT %R% optional(DGT)

# Create a unit pattern that matches an optional space, then one of YO, YR or MO.
unit <- optional(SPACE) %R% or("Y0", "YR", "MO")

# Test pattern with age then units
str_view(narr, pattern = age %R% unit)

HTML widgets cannot be represented in plain text (need html)

In [54]:
# Create a gender pattern that matches an optional space then M or F.Ç

# Use these patterns
age <- DGT %R% optional(DGT)
unit <- optional(SPC) %R% or("YO", "YR", "MO")

# Pattern to match gender
gender <- optional(SPC) %R% or("M", "F")

# Test pattern with age then units then gender
str_view(narr, pattern = age %R% unit %R% gender)

HTML widgets cannot be represented in plain text (need html)

In [58]:
# Use these patterns
age <- DGT %R% optional(DGT)
unit <- optional(SPC) %R% or("YO", "YR", "MO")
gender <- optional(SPC) %R% or("M", "F")

# Extract age, unit, gender
age_gender = str_extract(narr, pattern = age %R% unit %R% gender)

### Parsing age and gender into pieces
To finish up, you need to pull out the individual pieces and tidy them into usable variables.

There are a few ways you could get at one piece: you could extract out the piece you need, you could replace everything that isn't the piece you need with "", or you could try to split into the pieces you need. You'll try a few of these in this exercise and you'll see yet another way in the next chapter. For the first option, stringr has a nice convenience function, str_remove(), that works like str_replace() with replacement = "".

One benefit of building up your pattern in pieces is you already have patterns for each part that you can reuse now.

In [59]:
# Use str_extract with your age pattern to extract just the age from age_gender, then transform it to a number with as.numeric().
as.numeric(str_extract(age_gender, age))

[1] 19 31 82 33 10 53 13 14 55  5

In [60]:
# Create genders by using str_remove() with your age %R% unit pattern to replace everything except the gender with "".

# Replace age and units with ""
genders <- str_remove(age_gender, pattern = age %R% unit)

# Replace extra spaces
str_remove(genders, pattern = age %R% unit)

[1] "M"  "F"  "M"  "F"  "M"  " F" "F"  " M" "M"  "M"

In [61]:
# Numeric ages, from previous step
ages_numeric <- as.numeric(str_extract(age_gender, age))

# Get time_units by using str_extract() on age_gender with your unit pattern.

time_units <- str_extract(age_gender, pattern = unit)

# To know if the units are months or years we just need the first character after any spaces. 
# Use str_extract() on time_units with the pattern WRD to get time_units_clean.

time_units_clean <- str_extract(time_units, pattern = WRD)

# Turn ages in months to years
ifelse(time_units_clean == "Y", ages_numeric, ages_numeric/12)

[1] 19.000000 31.000000 82.000000 33.000000 10.000000 53.000000  1.083333
 [8] 14.000000 55.000000  5.000000